## **Feature selection is a process where you automatically select those features in your data that contribute most to the prediction variable or output in which you are interested. The benefits of performing feature selection before modeling your data are:**

* Avoid Overfitting: Less redundant data gives performance boost to the model and results in less opportunity to make decisions based on noise
* Reduces Training Time: Less data means that algorithms train faster



## 
* One common feature selection method that is used with text data is the Chi-Square feature selection. 
* The χ2 test is used in statistics to test the independence of two events.  
* More specifically in feature selection we use it to test whether the occurrence of a specific term and the occurrence of a specific class are independent. 

* For each feature (term), a corresponding high χ2 score indicates that the null hypothesis H0 of independence (meaning the document class has no influence over the term's frequency) should be rejected and the occurrence of the term and class are dependent. 
* In this case, we should select the feature for the text classification.

# **Import libraries**

In [0]:
# TODO: import pandas,numpy,labelbinarizer,selectkbest,countvectorizers
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer


# **Read the Datset**

In [0]:
#read csv
data = pd.read_csv("/content/iris.data",names=['sl','sw','pl','pw','class'])

In [0]:
data.head()

,sl,sw,pl,pw,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
# shape method gives the shape of dataframe
data.shape

(150, 5)

In [0]:
X = data.iloc[:,:4]
Y = data.iloc[:,-1]


In [0]:
X = X.values
print(X[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


# **STEPS¶**
* First compute the observed count for each class. This is done by building a contingency table from an input X (feature values) and y (class labels). 
*  Each entry i, j corresponds to some feature i and some class j, and holds the sum of the ith feature's values across all samples belonging to the class j.

* Note that although the feature values here are represented as frequencies, this method also works quite well in practice when the values are tf-idf values, since those are just weighted/scaled frequencies.

In [0]:
# binarize the output column,
# this makes computing the observed value a 
# simple dot product
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(Y)
print(transfomed_label[:5])


[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]]


In [0]:
y = transfomed_label

In [117]:
# our observed count for each class (the row)
# and each feature (the column)
observed = np.dot(y.T, X)
print(observed)

[[250.3 170.9  73.2  12.2]
 [296.8 138.5 213.   66.3]
 [329.4 148.7 277.6 101.3]]


In [118]:
# compute the probability of each class and the feature count; 
# keep both as a 2 dimension array using reshape
class_prob = y.mean(axis = 0).reshape(1, -1)
feature_count = X.sum(axis = 0).reshape(1, -1)
expected = np.dot(class_prob.T, feature_count)
print(expected)

[[292.16666667 152.7        187.93333333  59.93333333]
 [292.16666667 152.7        187.93333333  59.93333333]
 [292.16666667 152.7        187.93333333  59.93333333]]


### **We can do chi square test with an assumption that there is no biase between the columns. We can also do Chi_square test directly from the contigency table** 

### **find the Chi value and P value for each feature**
* chi-square scores - the scores are better if greater. 
* p-values - they are better if smaller.



In [119]:
#TODO : Find chisqscore for each feature value
chisq = (observed - expected) ** 2 / expected
chisq_score = chisq.sum(axis = 0)
print(chisq_score)

[ 10.81782088   3.59449902 116.16984746  67.24482759]


#**TODO : Cross check the same using Scikit learn chi2 function**


In [127]:
from sklearn.feature_selection import chi2
chi2score = chi2(X, y)
print(chi2score)

# 1st tuple is chisq value and 2nd tuple is pval

(array([ 10.81782088,   3.59449902, 116.16984746,  67.24482759]), array([4.47651499e-03, 1.65754167e-01, 5.94344354e-26, 2.50017968e-15]))


# **Feature Selection**

## **Select k best features using Chisquare as score fn**
**Scikit**-**learn** provides a **SelectKBest** class that can be used with a suite of different statistical tests. It will rank the features with the statistical test that we've specified and select the top k performing ones (meaning that these terms is considered to be more relevant to the task at hand than the others), where k is also a number that we can tweak.

In [130]:
#TODO : scikit selectkbest
chi2_features = SelectKBest(chi2, k = 2) 
X_kbest_features = chi2_features.fit_transform(X, y) 
  
# Reduced features 
print('Original feature number:', X.shape[1]) 
print('Reduced feature number:', X_kbest_features.shape[1]) 

Original feature number: 4
Reduced feature number: 2


# **Conclusion**